In [8]:
from pprint import pprint
import logging

from dotenv import load_dotenv
import openai
import pandas as pd
import pandas as pd 

from strategies import StrategyRating
from utils import StatsHolder

load_dotenv('../openai.env')
openai.api_key  =None# os.getenv('API_KEY')
logger = logging.getLogger()

In [9]:
stats = StatsHolder.from_csv(path_docs='resource/boxscoretraditionalv2/')

strategy_top_10 = StrategyRating(stats.players_stats, stats.calculus_columns, 10)
strategy_top_1 = StrategyRating(stats.players_stats, stats.calculus_columns, 1)
strategy_top_100 = StrategyRating(stats.players_stats, stats.calculus_columns, 100)

stats.add_strategy(strategy_top_10)
stats.add_strategy(strategy_top_1)
stats.add_strategy(strategy_top_100)

new_record = pd.read_csv('boxscoretraditionalv2_0_0012100003.csv') 
stats.add_record(new_record)

100%|██████████| 100/100 [00:01<00:00, 52.17it/s]
2023-10-31 17:49:21,042 - INFO - Added "StrategyRating Top-10" strategy
2023-10-31 17:49:21,043 - INFO - Added "StrategyRating Top-1" strategy
2023-10-31 17:49:21,044 - INFO - Added "StrategyRating Top-100" strategy
2023-10-31 17:49:21,060 - INFO - added game_id: 12100003.0
2023-10-31 17:49:21,459 - INFO - {'description': 'Top 10 players', 'values': {'in_top': {'FGA': {1628369}, 'FG3M': {203897, 1627741}, 'FT_PCT': {1627846}, 'DREB': {1628976}, 'REB': {1628976}, 'STL': {204456}}, 'out_top': {'FGA': {1630163}, 'FG3M': {203468, 1629670}, 'FT_PCT': {1629639}, 'DREB': {203999}, 'REB': {1628381}, 'STL': {1630163}}}}
2023-10-31 17:49:21,882 - INFO - {'description': 'Top 1 players', 'values': {'in_top': {}, 'out_top': {}}}
2023-10-31 17:49:22,267 - INFO - {'description': 'Top 100 players', 'values': {'in_top': {'MIN': {203082, 1630591, 1630181, 203935}, 'FGM': {1627759}, 'FGA': {1628964, 1630181, 1630591, 1627759}, 'FG3M': {202722, 203114, 163

In [11]:
for strategy in stats.strategies:#['StrategyRating Top-100'].show_data_strategy()
    print(strategy)
    pprint(stats.strategies[strategy].show_data_strategy())
    print('----------------------------------------------------------------------')

StrategyRating Top-10
{'description': 'Top 10 players',
 'values': {'in_top': {'DREB': {1628976},
                       'FG3M': {203897, 1627741},
                       'FGA': {1628369},
                       'FT_PCT': {1627846},
                       'REB': {1628976},
                       'STL': {204456}},
            'out_top': {'DREB': {203999},
                        'FG3M': {203468, 1629670},
                        'FGA': {1630163},
                        'FT_PCT': {1629639},
                        'REB': {1628381},
                        'STL': {1630163}}}}
----------------------------------------------------------------------
StrategyRating Top-1
{'description': 'Top 1 players', 'values': {'in_top': {}, 'out_top': {}}}
----------------------------------------------------------------------
StrategyRating Top-100
{'description': 'Top 100 players',
 'values': {'in_top': {'AST': {1630202},
                       'BLK': {1630170, 1630532, 1630591},
                       '

In [31]:
prompt = ''
  
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]
  
a = get_completion(prompt, model="gpt-3.5-turbo")
print(a)

Структура диалога:

I. Подготовка к игре
   A. Ознакомление с правилами
      1. Вопрос о знании правил
         - Игорь Охотников: "Ознакомлены ли вы с содержанием статей 'Об Игре' и 'О Клубе' и особенно с этическими правилами в статье 'Об Игре'?"
         - Яся Маратова: "1) да 2) не очень, сейчас все прочитаю"
      2. Подтверждение согласия с правилами
         - Игорь Охотников: "Согласны ли вы с этими правилами, готовы ли неукоснительно соблюдать их в ходе Игры? Есть ли какие-то сомнения или непонятные моменты в них?"
         - Яся Маратова: "Да, конечно согласна."
   B. Подтверждение слова чести
      - Игорь Охотников: "1) Вы согласны, что человек, подписавшийся под этими правилами, а затем злостно их нарушающий - человек непорядочный? 2) Даёте ли вы слово чести, что не будете так делать?"
      - Яся Маратова: "1. Да, конечно. 2. Да."
   C. Подтверждение понимания статьи о чести
      - Игорь Охотников: "Прочитайте еще раз статью о чести. Если вы согласны с ней, то повторите 